In [1]:
import numpy as np
import pandas as pd
import os
import cmath

In [4]:
# Reading bus data into pandas 

def buses(file):
    df_buses = pd.read_csv(file, sep=";")
    V = df_buses.V.to_numpy()
    delta = df_buses.Angle.to_numpy()
    P = df_buses.P_gen.to_numpy()
    Q = df_buses.Q_gen.to_numpy()
    return V,delta,P,Q


V,delta,P,Q = buses('Busdata.csv')

print(V)
print(delta)
print(P)
print(Q)

[1 0 1 0 0]
[0 0 0 0 0]
[ 1.  -0.6  0.  -0.6 -0.5]
[ 0.  -0.3  0.  -0.2 -0.4]


In [5]:
# Creating Y-bus matrix

def Ybus(file, shape):
    df_impedances = pd.read_csv(file, sep=";")
    print(df_impedances.head())
    Z_values = np.zeros((shape,shape), dtype=complex)
    Y_bus = np.zeros((shape,shape), dtype=complex)

    for x in range(df_impedances.shape[0]):
        from_line = df_impedances['From_line'][x]
        to_line = df_impedances['To_line'][x]
        Z_values[from_line-1][to_line-1] = df_impedances['R'][x] + df_impedances['X'][x]*1j

    
    for i in range(shape):
        for j in range(shape):
            if(Z_values[i][j] != 0. +0.j):
                Y_bus[i][j] = 1/Z_values[i][j]
            else:
                Y_bus[i][j] = Z_values[i][j]

    for x in range(df_impedances.shape[0]):
        from_line = df_impedances['From_line'][x]
        to_line = df_impedances['To_line'][x]
        
        Y_bus[from_line-1][from_line-1] += - 1/(df_impedances['R'][x] + df_impedances['X'][x]*1j)
        Y_bus[from_line-1][from_line-1] += 1/(2*(df_impedances['Full_line_B'][x]*1j))
        Y_bus[to_line-1][to_line-1] += 1/(2*(df_impedances['Full_line_B'][x]*1j))

    return Y_bus

In [8]:
def P_Calc(V, YBus, BusNum, delta, P):
    P_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if P[i] == 0:
            P_Calc[i] == 0
        else:
            for j in range (BusNum):
                P_Calc[i] = P_Calc[i] + (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) 
    return P_Calc

def Q_Calc(V, YBus, BusNum, delta, Q):
    Q_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if Q[i] == 0:
            Q_Calc[i] == 0
        else:
           for j in range (BusNum):
                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])
                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) 
    return Q_Calc



P_calc = (P_Calc(V, Ybus, 5, delta, P))
Q_calc = (Q_Calc(V, Ybus, 5, delta, Q))

print(P_calc)
print(Q_calc)

[-1.26428027  0.          0.          0.          0.        ]
[0. 0. 0. 0. 0.]


C:\Users\asbjo\AppData\Local\Temp\ipykernel_11348\888496469.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  P_Calc[i] = P_Calc[i] + (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
C:\Users\asbjo\AppData\Local\Temp\ipykernel_11348\888496469.py:19: ComplexWarning: Casting complex values to real discards the imaginary part
  Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])


In [7]:
Ybus = Ybus('impedances.csv', 5)

print(Ybus)

   From_line  To_line      R      X  Full_line_B
0          1        2  0.050  0.250       0.0500
1          2        3  0.025  0.125       0.0250
2          1        4  0.020  0.200       0.0333
3          2        4  0.020  0.200       0.0333
4          4        5  0.010  0.100       0.0200
[[-1.26428027-16.21836612j  0.76923077 -3.84615385j
   0.         +0.j          0.4950495  -4.95049505j
   0.         +0.j        ]
 [ 0.         +0.j         -2.03351104-32.37221227j
   1.53846154 -7.69230769j  0.4950495  -4.95049505j
   0.         +0.j        ]
 [ 0.         +0.j          0.         +0.j
   0.        -20.j          0.         +0.j
   0.         +0.j        ]
 [ 0.         +0.j          0.         +0.j
   0.         +0.j         -0.99009901-45.12903993j
   0.99009901 -9.9009901j ]
 [ 0.         +0.j          0.         +0.j
   0.         +0.j          0.         +0.j
   0.        -25.j        ]]
